In [1]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time
import nltk

# load the dataset
dataset = pd.read_csv('./data/train.csv')
dataset.head()

,Id,Popularity,Page content
0,0,-1,"<html><head><div class=""article-info""> <span c..."
1,1,1,"<html><head><div class=""article-info""><span cl..."
2,2,1,"<html><head><div class=""article-info""><span cl..."
3,3,-1,"<html><head><div class=""article-info""><span cl..."
4,4,-1,"<html><head><div class=""article-info""><span cl..."


In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    lem = WordNetLemmatizer()
    res = [lem.lemmatize(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res = []
for i in range(0, 100):
    res.append(get_category(dataset['Page content'][i]))

In [3]:
cats = res

In [2]:
from pytrends.request import TrendReq
from datetime import *
import time


In [6]:
#password = "e8512033" ; email = "stoneonetwo1203@gmail.com"
proxy_list = ["http://123.204.5.197:3128", "http://59.127.168.43:3128", "http://60.248.199.206:80", "http://1.164.248.90:8080"]
g_trends = TrendReq(hl='en-US', tz=360, timeout=(10,25), proxies=proxy_list, retries=2, backoff_factor=0.1)
def get_trend(word: str, t_start: str, t_end: str) -> list:
    g_trends.build_payload([word], timeframe = t_start+' '+t_end, geo='', gprop='')
    trend = g_trends.interest_over_time()
    if trend.size > 0:
        trend.columns = ['0', '1']
        #trend = trend.columns.get_loc()
        # trend.head()
        return trend['0'].values
    else: 
        print(word+" request fail.")
        return []

# test
print(get_trend('NFV', '2013-06-20', '2013-07-20'))
print(get_trend('NFV', '2013-06-20', '2013-07-20'))
# print(np.mean(get_trend('NFV', '2013-06-20', '2013-07-20'), axis=0))
# print(np.std(get_trend('NFV', '2013-06-20', '2013-07-20'), axis=0, ddof=1))
# print(np.std([1, 2, 3], ddof=1))

[ 92  89  51  65  99  83  88  94  64  83  82 100  79  75  99  81  35  44
  78  68  69  77  73  67  63  64  73  72  75  60  55]
Proxy error. Changing IP


ConnectTimeout: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22NFV%22%2C+%22time%22%3A+%222013-06-20+2013-07-20%22%2C+%22geo%22%3A+%22%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x0000029FDABF7320>, 'Connection to 1.164.248.90 timed out. (connect timeout=10)'))

In [9]:
def get_date(text: str) -> datetime:
    soup = BeautifulSoup(text)
    selector = "time"
    date = [i.text for i in soup.select(selector)][0]
    date = date.split()
    date = str(date[0] + ' ' + date[1])

    d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return d

In [10]:
def trend_num(word: str, pub_time: datetime):
    y_start = str(pub_time).split(' ')[0]
    y_end = str(pub_time + timedelta(days=14)).split(' ')[0]
    y_trend = get_trend(word, y_start, y_end)
    time.sleep(1)
    if len(y_trend) > 0:
        return np.sum(y_trend)
    else:
        return 0
    
def trend_rate(word: str, pub_time: datetime):
    now = float(trend_num(word, pub_time))
    past = float(trend_num(word, pub_time + timedelta(days=-15)))
    if past > 0:
        return float(trend_num(word, pub_time)) / float(trend_num(word, pub_time + timedelta(days=-31)))
    else:
        return 100
    
res = []
for i in range(10):
    print(i)
    date = get_date(dataset["Page content"][i])
    tmp = []
    for cat in cats[i]:
        tmp.append(trend_rate(cat, date))
    res.append(max(tmp))

0


ResponseError: The request failed: Google returned a response with code 429.

In [11]:
res

[]